In [34]:
import requests
import warnings 
from urllib.request import urlopen
import time 
from bs4 import BeautifulSoup
import re
from  tqdm import tqdm
from datetime import datetime
import re
import pandas as pd

# 1

## 1.1. Get the list of places

In [2]:
exception=[]  #In this list we store the exception in order to understand what is wrong.

#Since the link of the page contains number we use a for loop in order to change that number and so change the links.


for i in tqdm(range(1,401)):
    url='https://www.atlasobscura.com/places?page='+str(i)+'&sort=likes_count' #Move to the next web page 
    risposta=requests.get(url)                                                 #Request https protocolo        
    testo=BeautifulSoup(risposta.text,'html.parser')                           #Format the html
    
    try:
        first_div=testo.find_all("div",{"class":"row vue-js-bte-place-parent js-vue-component-wrap"}) #find the tag
        
        
        for j in range(0,len(first_div)):
            try:
                second_div=first_div[j].find_all("div",{"class":"col-md-4 col-sm-6 col-xs-12"})
    
                for k in range(0,len(second_div)):
                    try:
                        a=second_div[k].find("a").get("href").strip()    #extract the link of the page
                     
                        with open('links.txt',"a")as f:
                            f.write("https://www.atlasobscura.com"+str(a))  #write down on a txt file the complete link of the page.
                            f.write('\n')
                    except:
                        exception.append([i,j,k,risposta])      
            except:
                exception.append([i,j,None,risposta])   
    except:
        exception.append([i,None,None,risposta])
        
    
    time.sleep(1) #In ordere to avoid "429 Too Many Requests"
       
            
        

  0%|          | 1/400 [00:02<14:25,  2.17s/it]


KeyboardInterrupt: 

## 1.2. Crawl places

In [29]:
file = open('links.txt')
content = file.readlines()
for i in tqdm(range(0,len(content))):
    url=content[i].strip()    
    risposta=requests.get(url)                                               
    with open('HTML/html'+str(i)+'.txt',"a")as f:
        f.write(risposta.text)  #write down on a txt file the complete link of the page.
    time.sleep(1)


100%|██████████| 7200/7200 [2:18:44<00:00,  1.16s/it]  


## 1.3 Parse downloaded pages

### Function that extract data from single HTML

In [48]:
def estrazione(testo):  
    errors=[]
    #1 Place Name (to save as placeName): String.
    try:
        placeName=testo.find("h1",{"class":"DDPage__header-title"}).text.strip()
    except:
        placeName=" "
        errors.append(1)
    #---#
    #2 Place Tags (to save as placeTags): List of Strings.
    try:
        placeTags=[]
        ff2=testo.find("div",{"class":"item-tags col-xs-12"})
        ff2=ff2.find_all("span")
        for i in range(len(ff2)):
            placeTags.append(ff2[i].find("a").text.strip())
    except:
        placeTags=[" "]
        errors.append(2)



    #---#
    conta=testo.find_all("div",{"class","title-md item-action-count"})

    #3 # of people who have been there (to save as numPeopleVisited): Integer.
    try:
        numPeopleVisited=conta[0].text.strip()
    except:
        numPeopleVisited=" "
        errors.append(3)
    #---#
    #4 # of people who want to visit the place(to save as numPeopleWant): Integer.    
    try:
        numPeopleWant=conta[1].text.strip()
    except:
        numPeopleWant=" "
        errors.append(4)
    #---#
    #5 Description (to save as placeDesc): String. Everything from under the first image up to "know before you go" (orange frame on the example image).
    try:
        tag=testo.find("div",{"class":"DDP__body-copy"})
        tag=tag.find_all("p")
        placeDesc=""
        for i in range(0,len(tag)):
            placeDesc += str(tag[i].text)
    except:
        placeDesc=""
        errors.append(5)

    #6 Short Description (to save as placeShortDesc): String. Everything from the title and location up to the image (blue frame on the example image).
    placeShortDesc=""
    try:
        placeShortDesc2=testo.find("h3", {"class","DDPage__header-dek"}).text.strip()
    except:
        placeShortDesc2=" "
        errors.append(6.2)
    placeShortDesc += placeShortDesc2

    #7 Nearby Places (to save as placeNearby): Extract the names of all nearby places, but only keep unique values: List of Strings.
    placeNearby=[]
    try:
        place=testo.find("div",{"class":"DDPageSiderail__column grid-col-lg-4 grid-col-md-5"})
        place=place.find("div",{"class":"DDPageSiderail"})
        place=place.find("div", {"class":"DDPageSiderailRecirc"})
        place=place.find_all("a")
        for i in range(0,len(place)):
            luogo=place[i].find("div",{"class":"DDPageSiderailRecirc__item-title"})
            placeNearby.append(luogo.text.strip())
    except:
        placeNearby=[" "]
        errors.append(7)
    #8 Address of the place(to save as placeAddress): String.
    try:
        placeAddress=testo.find("div",{"class":"DDPageSiderail__column grid-col-lg-4 grid-col-md-5"})
        placeAddress=placeAddress.find("div",{"class":"DDPageSiderail"})
        placeAddress=placeAddress.find("aside", {"class":"DDPageSiderail__details"})
        placeAddress=placeAddress.find("address",{"class":"DDPageSiderail__address"})
        placeAddress=placeAddress.find("div").get_text(",").strip().rstrip(',').strip()
    except:
        placeAddress=" "
        errors.append(8)

    #9 Altitude and Longitude of the place's location(to save as placeAlt and placeLong): Integers
    try:
        coordinate=testo.find("div",{"class":"DDPageSiderail__column grid-col-lg-4 grid-col-md-5"})
        coordinate=coordinate.find("div",{"class":"DDPageSiderail"})
        coordinate=coordinate.find("aside", {"class":"DDPageSiderail__details"})
        coordinate=coordinate.find("div",{"class":"DDPageSiderail__coordinates js-copy-coordinates"}).text.strip().split()

        placeAlt=float(coordinate[0].replace(",",""))
        placeLong=float(coordinate[1].replace(",",""))
    except:
        placeAlt=" "
        placeLong=" "
        errors.append(9)

    #10 The username of the post editors (to save as placeEditors): List of Strings
    placeEditors=[]
    createdby=[]
    Editors=testo.find("div",{"id":"ugc-module"})
    Editors=Editors.find("div",{"class":"DDPContributors"})
    Editors=Editors.find_all("div",{"class":"ugc-editors"})

    try:
        for i in range(0,len(Editors)):
            tipo=Editors[i].find("h6").text.strip()
            if "edited by" == tipo.lower():
                try:
                    g=Editors[i].find("div",{"class":"DDPContributorsList"})
                    g=g.find("div",{"class":"js-editor-list hidden"})
                    g=g.find_all("li")
                    for  i in range(0,len(g)):
                        a=g[i].find("a").find("span").text.strip()
                        placeEditors.append(a)
                except:
                    placeEditors=[" "]
                    #errors.append(10.1)
                    
            elif "added by"==tipo.lower():
                try:
                    creator=Editors[i].find("div",{"class":"DDPContributorsList"})
                    createdby.append(creator.find("a").text)
                except:
                    createdby=[" "]
                   # errors.append(10.2)
    except:
        createdby=[" "]
        placeEditors=[" "] 
        errors.append(10)
    placeEditors=",".join(placeEditors)
    createdby=",".join(createdby)


    #11 Post publishing date (to save as placePubDate): datetime.
    try:
        placePubDate=testo.find("div",{"id":"ugc-module"})
        placePubDate=placePubDate.find("div",{"class":"DDPContributors"})
        placePubDate=placePubDate.find("div",{"class":"DDPContributor__name"}).text.strip()
        placePubDate=datetime.strptime(placePubDate, "%B %d, %Y").date() #year, month, day
    except:
        placePubDate=" "
        errors.append(11)

    #12 The names of the lists that the place was included in (to save as placeRelatedLists): List of Strings.
    placeRelatedLists=[]
    try:
        o=testo.find_all("div",{"class":"card-grid CardRecircSection__card-grid js-inject-gtm-data-in-child-links"})
        o=o[2].find_all("div",{"class":"CardWrapper js-CardWrapper"})
        for i in range(len(o)):
            m=o[i].find("h3").text.strip()
            placeRelatedLists.append(m)
    except:
        placeRelatedLists=[" "]
        errors.append(12)

    #13 The names of the related places (to save as placeRelatedPlaces): List of Strings.
    placeRelatedPlaces=[]
    try:
        o=testo.find_all("div",{"class":"card-grid CardRecircSection__card-grid js-inject-gtm-data-in-child-links"})
        o=o[1].find_all("div",{"class":"CardWrapper js-CardWrapper"})
        for i in range(len(o)):
            m=o[i].find("h3").text.strip()
            placeRelatedPlaces.append(m)
    except:
        placeRelatedPlaces=[" "]
        errors.append(13)

    #14 The URL of the page of the place (to save as placeURL):String
    try:
        placeURL=testo.find("link",{"rel":"canonical"}).get("href")
    except:
        placeURL=" "
        errors.append(14)
    
    

    try:
        link=testo.find("div",{"class":"DDPage__header-place-location"})
        link=link.find("a").get("href").strip()
    except:
        link=" "
       
    
    d={"placeName":placeName,"placeTags": ",".join(placeTags),"numPeopleVisited":int(numPeopleVisited),"numPeopleWant":int(numPeopleWant),"placeDesc":placeDesc,"placeShortDesc":placeShortDesc,
    "placeNearby":",".join(placeNearby),"placeAddress":placeAddress.strip(),"placeAlt":placeAlt,"placeLong":placeLong,"createdby":createdby,"placeEditors":placeEditors,"placePubDate":placePubDate,
    "placeRelatedLists":",".join(placeRelatedLists),"placeRelatedPlaces":",".join(placeRelatedPlaces),"placeURL":placeURL}

    return  d,errors      

## Create dataframe

#### Create 7200 tsv files, place_i.tsv and one csv file

In [49]:
anomalie={}
for i in tqdm(range(0,7200)):
    
    with open('HTML/html'+str(i)+'.txt') as f:
        testo = f.read()

    testo=BeautifulSoup(testo,'html.parser') 
    d,errori=estrazione(testo)
    
    if len(errori) != 0:
        anomalie[str(i)]=errori
        
    df=pd.DataFrame(d,index=[0])
    df.to_csv("TSV/place_"+str(i)+".tsv",sep="\t")
    if i==0 :
      dfff=pd.DataFrame(d,index=[i])
    else:
      dataframe=pd.DataFrame(d,index=[i])
      dfff=dfff.append(dataframe)
        
dfff.to_csv("places2.csv",sep="\t")

100%|██████████| 7200/7200 [22:56<00:00,  5.23it/s]


#### Overview of the dataset

In [39]:
df=pd.read_csv("places2.csv",delimiter="\t")
df.shape

(7200, 17)

#### Analysis of the errors

In [41]:
with open('anomalie.txt',"w")as f:
    for i in anomalie.keys():
        f.write(i)
        f.write(str(anomalie[i]))
        f.write("\n")

In [40]:
df.placeURL[df.index==6037]

6037    https://www.atlasobscura.com/places/shangri-la-botanical-gardens-nature-center
Name: placeURL, dtype: object

In [43]:
for i in anomalie.keys():
    print(anomalie[i],i)

[12] 79
[12] 84
[12] 91
[12] 92
[12] 100
[12] 102
[12] 107
[12] 109
[12] 113
[12] 115
[12] 120
[12] 124
[12] 133
[12] 134
[12] 135
[12] 136
[12] 141
[12] 144
[12] 145
[12] 149
[12] 151
[12] 152
[12] 154
[12] 159
[12] 161
[12] 162
[12] 167
[12] 172
[12] 174
[12] 176
[12] 181
[12] 184
[12] 189
[12] 190
[12] 198
[12] 204
[12] 205
[12] 206
[12] 207
[12] 208
[12] 210
[12] 221
[12] 222
[12] 223
[12] 226
[12] 228
[12] 230
[12] 231
[12] 232
[12] 234
[12] 236
[12] 239
[12] 245
[12] 248
[12] 250
[12] 251
[12] 256
[12] 260
[12] 261
[12] 263
[12] 264
[12] 269
[12] 272
[12] 273
[12] 274
[12] 279
[12] 282
[12] 285
[12] 286
[12] 287
[12] 290
[12] 294
[12] 296
[12] 297
[12] 299
[12] 300
[12] 301
[12] 302
[12] 304
[12] 308
[12] 309
[12] 311
[12] 312
[12] 313
[12] 315
[12] 317
[12] 326
[12] 327
[12] 328
[12] 329
[12] 333
[12] 334
[12] 335
[12] 338
[12] 342
[12] 343
[12] 344
[12] 346
[12] 352
[12] 353
[12] 354
[12] 357
[12] 360
[12] 361
[12] 362
[12] 365
[12] 366
[12] 367
[12] 372
[12] 375
[12] 377
[12] 

**Question 2**

In [73]:
import nltk
import os
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import PorterStemmer

In [100]:
dir_path='TSV'
porter = PorterStemmer()
placesDf=pd.DataFrame()
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)
for path in os.listdir(dir_path):
    print(path)
    if path !='.ipynb_checkpoints':
        df = pd.read_csv(dir_path+'/'+path,sep='\t',usecols = ['placeName','placeDesc','placeURL'])
        df['placeName'] = df['placeName'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
        df['placeName'] = df['placeName'].str.replace('[^\w\s]','')
        for i in df['placeName']:
            df['placeName']=stemSentence(i)
        df['placeDesc'] = df['placeDesc'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
        df['placeDesc'] = df['placeDesc'].str.replace('[^\w\s]','')
        for i in df['placeDesc']:
            df['placeDesc']=stemSentence(i)
        df.to_csv('TSV_CHANGES/'+path,index=False)
        placesDf = pd.concat([placesDf,df])

place_5956.tsv
place_4490.tsv
place_6287.tsv
place_7199.tsv
place_387.tsv
place_2787.tsv
place_3499.tsv
place_2793.tsv
place_6293.tsv
place_393.tsv
place_5942.tsv
place_4484.tsv
place_2963.tsv
place_2977.tsv
place_344.tsv
place_6244.tsv


/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipykernel_1719/526328499.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['placeName'] = df['placeName'].str.replace('[^\w\s]','')
/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipykernel_1719/526328499.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df['placeDesc'] = df['placeDesc'].str.replace('[^\w\s]','')
/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipykernel_1719/526328499.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['placeName'] = df['placeName'].str.replace('[^\w\s]','')
/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipykernel_1719/526328499.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df['placeDesc'] = df['placeDesc'].str.replace('[^\w\s]','')
/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipy

place_2022.tsv
place_5995.tsv
place_4453.tsv
place_4335.tsv
place_2744.tsv
place_6522.tsv
place_422.tsv
place_6536.tsv
place_1259.tsv
place_436.tsv
place_2988.tsv
place_4321.tsv
place_2750.tsv
place_5759.tsv
place_2036.tsv
place_3328.tsv
place_5981.tsv
place_4447.tsv
place_350.tsv
place_6250.tsv
place_5771.tsv


/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipykernel_1719/526328499.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df['placeDesc'] = df['placeDesc'].str.replace('[^\w\s]','')
/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipykernel_1719/526328499.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['placeName'] = df['placeName'].str.replace('[^\w\s]','')
/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipykernel_1719/526328499.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df['placeDesc'] = df['placeDesc'].str.replace('[^\w\s]','')
/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipykernel_1719/526328499.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['placeName'] = df['placeName'].str.replace('[^\w\s]','')
/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipy

place_3300.tsv
place_7166.tsv
place_378.tsv
place_1517.tsv
place_6278.tsv
place_1271.tsv
place_4309.tsv
place_3466.tsv
place_2778.tsv
place_5017.tsv
place_3472.tsv
place_5003.tsv
place_1265.tsv
place_7172.tsv
place_1503.tsv
place_5765.tsv
place_3314.tsv
place_3843.tsv
place_2585.tsv
place_185.tsv
place_1932.tsv


...

/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipykernel_1719/526328499.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['placeName'] = df['placeName'].str.replace('[^\w\s]','')
/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipykernel_1719/526328499.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df['placeDesc'] = df['placeDesc'].str.replace('[^\w\s]','')
/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipykernel_1719/526328499.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['placeName'] = df['placeName'].str.replace('[^\w\s]','')
/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipykernel_1719/526328499.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df['placeDesc'] = df['placeDesc'].str.replace('[^\w\s]','')
/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipy

place_3319.tsv
place_4476.tsv
place_5768.tsv
place_2007.tsv
place_1526.tsv
place_6249.tsv
place_7157.tsv
place_349.tsv
place_3331.tsv
place_5998.tsv
place_5740.tsv
place_2749.tsv
place_5026.tsv
place_2991.tsv
place_4338.tsv
place_3457.tsv
place_1240.tsv
place_1254.tsv
place_5032.tsv
place_2985.tsv


/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipykernel_1719/526328499.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['placeName'] = df['placeName'].str.replace('[^\w\s]','')
/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipykernel_1719/526328499.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df['placeDesc'] = df['placeDesc'].str.replace('[^\w\s]','')
/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipykernel_1719/526328499.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['placeName'] = df['placeName'].str.replace('[^\w\s]','')
/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipykernel_1719/526328499.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df['placeDesc'] = df['placeDesc'].str.replace('[^\w\s]','')
/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipy

place_3443.tsv
place_3325.tsv
place_5754.tsv
place_1532.tsv
place_7143.tsv
place_5967.tsv
place_5973.tsv
place_5783.tsv
place_7194.tsv
place_1283.tsv
place_2952.tsv
place_3494.tsv
place_2946.tsv
place_3480.tsv
place_1297.tsv
place_7180.tsv
place_5797.tsv
place_4489.tsv


/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipykernel_1719/526328499.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['placeName'] = df['placeName'].str.replace('[^\w\s]','')
/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipykernel_1719/526328499.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df['placeDesc'] = df['placeDesc'].str.replace('[^\w\s]','')
/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipykernel_1719/526328499.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['placeName'] = df['placeName'].str.replace('[^\w\s]','')
/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipykernel_1719/526328499.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df['placeDesc'] = df['placeDesc'].str.replace('[^\w\s]','')
/var/folders/5b/bml_986d6rn1rxlht9v8rb4w0000gn/T/ipy

In [79]:
placedDescDf=placesDf[['placeDesc']]

In [95]:
termsDf = pd.DataFrame( columns=['term_id','term'])

In [96]:
counter = 0
for i in placedDescDf['placeDesc']:
    lst = i.split()
    for j in lst:
        if j not in termsDf.values:
            termsDf.loc[len(termsDf.index)] = [counter, j]
            counter+=1
    

In [98]:
termsDf.to_csv('vocabulary.csv',index=False)

In [101]:
from collections import defaultdict


In [130]:
d = defaultdict(list)


In [ ]:
for i in range(len(termsDf)):
    word = termsDf.iloc[i]['term']
    for path in os.listdir('TSV_CHANGES'):
        docsDf = pd.read_csv(dir_path+'/'+path,sep='\t',usecols = ['placeDesc'])
        s = docsDf.iloc[0]['placeDesc']
        if word in s:
            d[i].append(path)
    

In [ ]:
d